<a href="https://colab.research.google.com/github/chw8207/Transformer/blob/main/Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00


In [5]:
import tensorflow as tf
from transformers import pipeline

### GPU 설정

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13828851265908586718
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15208677376
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6542572143202177879
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
xla_global_id: 416903419
]


### 방법론

### 방법0: 시행착오

In [8]:
# 디폴트 모델과 토크나이저
nlp_qa = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [9]:
sequence = """The traffic began to slow down on Pioneer Boulevard in Los Angeles,
making it difficult to get out of the city. However, WBGO was playing some cool jazz,
and the weather was cool, making it rather pleasant to be making it out of the city
on this Friday afternoon. Nat King Cole was singing as Jo and Maria slowly made
their way out of LA and drove toward Barstow. They planned to get to Las Vegas
early enough in the evening to have a nice dinner and go see a show.
"""


In [10]:
nlp_qa(context=sequence, question='Where is Pioneer Boulevard?')

{'score': 0.9879735112190247, 'start': 55, 'end': 66, 'answer': 'Los Angeles'}

#### 방법 1: NER firts

In [11]:
# 질문을 찾기 위해 NER 사용하기
nlp_ner = pipeline('ner')
print(nlp_ner(sequence))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-LOC', 'score': 0.9735258, 'index': 8, 'word': 'Pioneer', 'start': 34, 'end': 41}, {'entity': 'I-LOC', 'score': 0.9944825, 'index': 9, 'word': 'Boulevard', 'start': 42, 'end': 51}, {'entity': 'I-LOC', 'score': 0.9995776, 'index': 11, 'word': 'Los', 'start': 55, 'end': 58}, {'entity': 'I-LOC', 'score': 0.99956936, 'index': 12, 'word': 'Angeles', 'start': 59, 'end': 66}, {'entity': 'I-ORG', 'score': 0.9919844, 'index': 26, 'word': 'W', 'start': 122, 'end': 123}, {'entity': 'I-ORG', 'score': 0.9907504, 'index': 27, 'word': '##B', 'start': 123, 'end': 124}, {'entity': 'I-ORG', 'score': 0.9884582, 'index': 28, 'word': '##G', 'start': 124, 'end': 125}, {'entity': 'I-ORG', 'score': 0.9722683, 'index': 29, 'word': '##O', 'start': 125, 'end': 126}, {'entity': 'I-PER', 'score': 0.99668807, 'index': 59, 'word': 'Nat', 'start': 267, 'end': 270}, {'entity': 'I-PER', 'score': 0.9976484, 'index': 60, 'word': 'King', 'start': 271, 'end': 275}, {'entity': 'I-PER', 'score': 0.9986173, 'ind

In [13]:
# 위치 엔터티 질문
nlp_qa = pipeline('question-answering')
print('Question 1.', nlp_qa(context=sequence, question='Where is Pioneer Boulevard?'))
print('Question 2.', nlp_qa(context=sequence, question='Where is Los Angeles located?'))
print('Question 3.', nlp_qa(context=sequence, question='Where is LA?'))
print('Question 4.', nlp_qa(context=sequence, question='Where is Barstow?'))
print('Question 5.', nlp_qa(context=sequence, question='Where is Las Vegas located?'))

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Question 1. {'score': 0.9879735112190247, 'start': 55, 'end': 66, 'answer': 'Los Angeles'}
Question 2. {'score': 0.9875389337539673, 'start': 34, 'end': 51, 'answer': 'Pioneer Boulevard'}
Question 3. {'score': 0.5090532302856445, 'start': 55, 'end': 66, 'answer': 'Los Angeles'}
Question 4. {'score': 0.36954542994499207, 'start': 391, 'end': 400, 'answer': 'Las Vegas'}
Question 5. {'score': 0.2183990627527237, 'start': 359, 'end': 366, 'answer': 'Barstow'}


In [14]:
# 개인 엔터티 질문
nlp_qa = pipeline('question-answering')
nlp_qa(context=sequence, question='Who was singing?')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9653673768043518,
 'start': 267,
 'end': 280,
 'answer': 'Nat King Cole'}

In [15]:
nlp_qa(context=sequence, question='Who was going to Las Vegas?')

{'score': 0.4316229820251465,
 'start': 267,
 'end': 280,
 'answer': 'Nat King Cole'}

#### SRL first

In [16]:
nlp_qa(context=sequence, question='Who are they?')

{'score': 0.8486901521682739,
 'start': 296,
 'end': 308,
 'answer': 'Jo and Maria'}

In [17]:
nlp_qa(context=sequence, question='Who drove to Las Vegas?')

{'score': 0.35941487550735474,
 'start': 267,
 'end': 308,
 'answer': 'Nat King Cole was singing as Jo and Maria'}

#### ELECTRA를 사용한 질문-답변

In [18]:
print(nlp_qa.model)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      